# Save to netcdf surface currents, wind speeds and salinity
This script is based on Ashu's "Make_profiles.ipynb" notebook in "climatology_analysis_notebooks"

In [2]:
import os
import xarray as xr
import numpy as np
from dateutil.parser import parse
# import the file path location from the forcing_paths.py script
import importlib.util
spec = importlib.util.spec_from_file_location("forcing_paths.py", "/ocean/abhudia/MEOPAR/analysis-ashutosh/scripts/make-hdf5/forcing_paths.py")
paths = importlib.util.module_from_spec(spec)
spec.loader.exec_module(paths)

In [3]:
def timer(func):
    """Decorator function for timing function calls
    """
    def f(*args, **kwargs):
        beganat = time.time()
        rv = func(*args, *kwargs)
        elapsed = time.time() - beganat
        hours = int(elapsed / 3600)
        mins = int((elapsed - (hours*3600))/60)
        secs = int((elapsed - (hours*3600) - (mins*60)))
        print('\nTime elapsed: {}:{}:{}\n'.format(hours, mins, secs))
        return rv
    return f

## Define record length, grid location, and output file to use for graphic(s) 

In [4]:
# time period to plot up
start_time = '26 nov 2017'
end_time = '14 dec 2017'
#start_time = '30 may 2015'
#end_time = '22 jun 2015'

# time period of model simulations to overlay on longer time series
start_model = '1 dec 2017'
end_model = '7 dec 2017'
#start_model = '5 dec 2017'
#end_model = '12 dec 2017'

# SalishSeaCast and HRDPS grid indexing from [Ashu's "Pick Grid Points" script](https://nbviewer.jupyter.org/urls/bitbucket.org/midoss/analysis-ashutosh/raw/default/climatology_analysis_notebooks//Pick%20grid%20points.ipynb) 
# grid indexing for SalishSeaCast (note: I think of grid indexing as [y,x] and use this naming convention even though I'm not sure whether python orients in this way. )
loc = 'SOG' 
ssc_y = 474 
ssc_x = 252

# grid indexing for HRDPS
hrdps_y = 143 
hrdps_x = 141

# output netcdf to save data to
output_current_netcdf = loc + '_current_timeseries_' + ''.join(start_time.split()) + '_' + ''.join(end_time.split()) + '.nc'
output_wind_netcdf    = loc + '_wind_timeseries_' + ''.join(start_time.split()) + '_' + ''.join(end_time.split()) + '.nc'
output_salty_netcdf   = loc + '_salinity_timeseries_' + ''.join(start_time.split()) + '_' + ''.join(end_time.split()) + '.nc'

### Surface currents

In [5]:
u_paths = paths.salishseacast_paths(parse(start_time), parse(end_time), '/results2/SalishSea/nowcast-green.201806/', 'grid_U')
v_paths = paths.salishseacast_paths(parse(start_time), parse(end_time), '/results2/SalishSea/nowcast-green.201806/', 'grid_V')

### Salinity

In [6]:
salty_paths = paths.salishseacast_paths(parse(start_time), parse(end_time), '/results2/SalishSea/nowcast-green.201806/', 'grid_T')

### HRDPS

In [7]:
wind_paths = paths.hrdps_paths(parse(start_time), parse(end_time), '/results/forcing/atmospheric/GEM2.5/operational/')

## Define grid location from analysis-ashutosh/climatology_analysis_notebooks/Pick grid points.ipynb, we have the locations of the three points chosen as: ((y,x) order)

In [8]:
SoG=(np.array([ssc_y]), np.array([ssc_x]))

### Concatenate surface current vectors

In [9]:
# create u-velocity vector
first = True
for file_path in u_paths:
    f = xr.open_dataset(file_path).isel(depthu = 0).vozocrtx
    u_now = f.isel(y = ssc_y, x = ssc_x)
    time_now = f.time_centered
    if first is True:
        u_velocity = u_now
        time = time_now
        first = False
    else:
        u_velocity = xr.concat((u_velocity, u_now), dim = 'time_counter')
        time = xr.concat((time, time_now), dim = 'time_counter')

# create v-velocity vector  
# we will use "time" from the u-velocity loop and won't repeat here
first = True
for file_path in v_paths:
    f = xr.open_dataset(file_path).isel(depthv = 0).vomecrty
    v_now = f.isel(y = ssc_y, x = ssc_x)
    if first is True:
        v_velocity = v_now
        first = False
    else:
        v_velocity = xr.concat((v_velocity, v_now), dim = 'time_counter')
        

### Calculate current speed

In [10]:
u = u_velocity.values
v = v_velocity.values
U = (u**2 + v**2)**0.5

### Save current speed to file

In [11]:
# finally, stitch them together and turn the profile into a netcdf file
# The syntax ('time',U) identifies "U" as the "Data variable" dependent on time
current_speed = xr.Dataset(data_vars={'uspd': ('time_counter',U)}, coords={'time_counter': time})

# remove netcdf file if it already exists
if os.path.exists(output_current_netcdf):
    os.remove(output_current_netcdf)

# write to netcdf
current_speed.to_netcdf(output_current_netcdf, format = 'NETCDF4',engine = 'netcdf4')
print(output_current_netcdf)

SOG_current_timeseries_26nov2017_14dec2017.nc


In [14]:
# u-wind speed
first = True
for file_path in wind_paths:
    f = xr.open_dataset(file_path).u_wind
    u_now = f.isel(y = hrdps_y, x = hrdps_x)
    time_now = f.time_counter
    #time_now = f.time_centered
    if first is True:
        u_wind = u_now
        time = time_now
        first = False
    else:
        u_wind = xr.concat((u_wind, u_now), dim = 'time_counter')
        time = xr.concat((time, time_now), dim = 'time_counter')
        
# v-wind speed
first = True
for file_path in wind_paths:
    f = xr.open_dataset(file_path).v_wind
    v_now = f.isel(y = hrdps_y, x = hrdps_x)
    if first is True:
        v_wind = v_now
        first = False
    else:
        v_wind = xr.concat((v_wind, v_now), dim = 'time_counter')

In [13]:
f


<xarray.DataArray 'u_wind' (time_counter: 24, y: 266, x: 256)>
[1634304 values with dtype=float32]
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2017-11-26 ...
  * x             (x) float64 0.0 2.5e+03 5e+03 7.5e+03 1e+04 1.25e+04 ...
  * y             (y) float64 0.0 2.5e+03 5e+03 7.5e+03 1e+04 1.25e+04 ...
Attributes:
    short_name:   UGRD_10maboveground
    long_name:    U-Component of Wind
    level:        10 m above ground
    units:        m/s
    coordinates:  longitude latitude

### Calculate wind speed

In [15]:
u = u_wind.values
v = v_wind.values
U_wind = (u**2 + v**2)**0.5

### Save current speed to file

In [16]:
# finally, stitch them together and turn the profile into a netcdf file
# The syntax ('time',U) identifies "U" as the "Data variable" dependent on time
wind_speed = xr.Dataset(data_vars={'wspd': ('time_counter',U_wind)}, coords={'time_counter': time})

# remove netcdf file if it already exists
if os.path.exists(output_wind_netcdf):
    os.remove(output_wind_netcdf)

# write to netcdf
wind_speed.to_netcdf(output_wind_netcdf, format = 'NETCDF4',engine = 'netcdf4')
print(output_wind_netcdf)

SOG_wind_timeseries_26nov2017_14dec2017.nc


### Concatenate sea surface salinity (SSS) values

In [17]:
# create u-velocity vector
first = True
for file_path in salty_paths:
    f = xr.open_dataset(file_path).isel(deptht = 0).vosaline
    s_now = f.isel(y = ssc_y, x = ssc_x)
    time_now = f.time_counter
    if first is True:
        salinity = s_now
        time = time_now
        first = False
    else:
        salinity = xr.concat((salinity, s_now), dim = 'time_counter')
        time = xr.concat((time, time_now), dim = 'time_counter')

### Save SSS to file

In [18]:
# finally, stitch them together and turn the profile into a netcdf file
# The syntax ('time',U) identifies "U" as the "Data variable" dependent on time
salty = xr.Dataset(data_vars={'salinity': ('time_counter',salinity)}, coords={'time_counter': time})

# remove netcdf file if it already exists
if os.path.exists(output_salty_netcdf):
    os.remove(output_salty_netcdf)

# write to netcdf
salty.to_netcdf(output_salty_netcdf, format = 'NETCDF4',engine = 'netcdf4')
print(output_salty_netcdf)

SOG_salinity_timeseries_26nov2017_14dec2017.nc
